In [5]:
import pandas as pd
import json
import pathlib
with open('all_train.csv','r') as f:
    df_train = pd.read_csv(f,header=None,names=['path','p1','p2','p3','p4','obj'],usecols=['path','obj'])
    # print((df_train.iloc[-1]))
for index in df_train.index:
    parts = pathlib.Path(df_train.loc[index,'path']).parts[1:]
    path = pathlib.Path(*parts)
    df_train.loc[index,'path'] = str(path)

question_template = 'how many ##obj are there?'
tmp_path =  ''
json_data = []
obj_list = []
obj_num = 0
for i in range(len(df_train)):
    path = df_train.iloc[i]['path']
    obj = df_train.iloc[i]['obj']
    if path != tmp_path:
        #新图像
        if len(obj_list) > 0 :
            new_json_pair = {
                'question':question_template.replace('##obj',obj_list[-1]),
                'answer':str(obj_num),
                'image':tmp_path,
                'dataset':'pre'
            }
            json_data.append(new_json_pair)
        obj_num = 1
        obj_list = [obj]
        tmp_path = path
    elif obj not in obj_list:
        #新问题
        if len(obj_list) > 0  :
            new_json_pair = {
                'question':question_template.replace('##obj',obj_list[-1]),
                'answer':str(obj_num),
                'image':tmp_path,
                'dataset':'pre'
            }
            json_data.append(new_json_pair)
        obj_num = 1
        obj_list.append(obj)
    else:
        obj_num += 1
if len(obj_list) > 0  :
            new_json_pair = {
                'question':question_template.replace('##obj',obj_list[-1]),
                'answer':str(obj_num),
                'image':tmp_path,
                'dataset':'pre'
            }
json_data.append(new_json_pair)
 



In [6]:
print(len(json_data))
with open('json_train.json','w') as f:
    json.dump(json_data,f)

22516


In [4]:
import pandas as pd
import json
import random
with open('json_train.json','r') as f:
   data = json.load(f)
length = len(data)
train_len = int(0.8*length)
random.shuffle(data)
with open('pre_train.json','w') as f:
    json.dump(data[:train_len],f)
        
with open('pre_test.json','w') as f: 
    json.dump(data[train_len:],f)  